In [1]:
import os
import glob
import cv2

import numpy as np

In [2]:
class DriverDistractionHelper:
    
    def __init__(self, img_rows, img_cols, color_type=1):        
        x_manual_tests = self.read_and_normalize_manual_test_data(img_rows, 160, color_type)
        self.x_manual_tests = x_manual_tests
    
    def read_and_normalize_manual_test_data(self, img_rows, img_cols, color_type=1):
        cache_path = os.path.join('cache', 'test_r_' + str(img_rows) + '_c_' + str(img_cols) + '_t_' + str(color_type) + '.dat')
        if not os.path.isfile(cache_path) or use_cache == 0:
            test_data = self.load_manual_test(img_rows, img_cols, color_type)
            self.cache_data((test_data), cache_path)
        else:
            print('Restore test from cache!')
            (test_data, test_id) = restore_data(cache_path)

        test_data = np.array(test_data, dtype=np.uint8)
        test_data = test_data.reshape(test_data.shape[0], color_type, img_rows, img_cols)
        test_data = test_data.astype('float32')
        test_data /= 255
        print('Test shape:', test_data.shape)
        print(test_data.shape[0], 'test samples')
        return test_data
    
    def load_manual_test(self, img_rows, img_cols, color_type=1):
        print('Read manual test images')
        path = os.path.join('..', 'input', 'test', '*.jpg')
        files = glob.glob(path)
        X_test = []
        total = 0
        for fl in files:
            flbase = os.path.basename(fl)
            img = self.get_image(fl, img_rows, img_cols, color_type)
            X_test.append(img)
            total += 1
            if(total == 10):
                break
        return X_test
    
    def cache_data(self, data, path):
        if os.path.isdir(os.path.dirname(path)):
            file = open(path, 'wb')
            pickle.dump(data, file)
            file.close()
        else:
            print('Directory doesnt exists')
    
    def get_driver_data(self):
        dr = dict()
        path = os.path.join('..', 'input', 'driver_imgs_list.csv')
        print('Read drivers data')
        f = open(path, 'r')
        line = f.readline()
        while (1):
            line = f.readline()
            if line == '':
                break
            arr = line.strip().split(',')
            dr[arr[2]] = arr[0]
        f.close()
        return dr
    
    def get_image(self, path, img_rows, img_cols, color_type=1):
        # Load as grayscale
        if color_type == 1:
            img = cv2.imread(path, 0)
        elif color_type == 3:
            img = cv2.imread(path)
        return cv2.resize(img, (img_cols, img_rows))

In [3]:
ddh = DriverDistractionHelper(229, 229, 3)

Read manual test images
Directory doesnt exists
('Test shape:', (10, 3, 229, 160))
(10, 'test samples')


In [4]:
from keras.applications import xception

Using TensorFlow backend.


In [5]:
x_manual_test = xception.preprocess_input(ddh.x_manual_tests)
xception_model = xception.Xception(include_top=False, weights='imagenet', pooling='avg')

  647168/83683744 [..............................] - ETA: 1:07:38

KeyboardInterrupt: 

In [31]:
x = x_manual_test.reshape(10, 3*229*160)
x.shape

(10, 109920)

In [16]:
from sklearn.externals import joblib
xception_log_reg = joblib.load('models/logistic_regression_xception.pkl')

In [32]:
y_pred = xception_log_reg.predict(x)

ValueError: X has 109920 features per sample; expecting 2048